In [200]:
#Setup
import pandas as pd
import altair as alt
from pathlib import Path
import numpy as np
import statsmodels.api as sm
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [201]:
parent_path = str(Path().resolve().parent.parent) + "/"
data_path = "data/"
processed_path = "processed/"
processed_data_file = "processed_data.csv"

In [202]:
df = pd.read_csv(parent_path + data_path + processed_path + processed_data_file)

In [203]:
purchases = df[~df['InvoiceNo'].str.contains('C')]
returns = df[df['InvoiceNo'].str.contains('C')]
# Delet the numbners of returns out of the purchases
purchases = purchases[~purchases['InvoiceNo'].isin(returns['InvoiceNo'])]
purchases_count = purchases.shape[0]
returns_count = returns.shape[0]
# Create a dataframe with the counts
data = pd.DataFrame({'Category': ['Purchases', 'Returns'], 'Count': [purchases_count, returns_count]})



# Chart Beschreibung

Um einen klaren Fokus zu schaffen, habe ich den Rahmen des Diagramms entfernt und die Legende durch den Text im Chart ersetzt. Das besonders hervorgehobene Stück ist durch ein leuchtendes Rot hervorgehoben und zieht so die Aufmerksamkeit des Betrachters auf sich. Dabei habe ich bei der Gestaltung auf Einfachheit geachtet und das einheitliche Grau-Rot Farbschema verwendet.

In [245]:
chart = alt.Chart(data).encode(
    theta=alt.Theta("Count:Q"),
    color=alt.condition(
        alt.FieldOneOfPredicate("Category", ["Returns"]),
        alt.value("darkred"),     
        alt.value("grey")     
    )
).properties(
    title={"text":["Distribution of returns versus keept purchases"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"]},
    width=550,
    height=350
)

pie = chart.mark_arc(outerRadius=130,angle=270)

text_returns = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=700,
    color="darkred"
).encode(
    x=alt.value(55),  # pixels from left
    y=alt.value(173),  # pixels from top
    text=alt.value("Returns: 2%")
)

text_purchases = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=700,
    color="grey"
).encode(
    x=alt.value(410),  # pixels from left
    y=alt.value(173),  # pixels from top
    text=alt.value("Purchases: 98%")
)

alt.layer(pie, text_returns, text_purchases).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=16
)

alt.LayerChart(...)

# Conclusion 

Eine Retourenrate von 2% ist in der Welt des Online-Handels relativ gering und hat normalerweise keinen signifikanten Einfluss auf das Geschäft. In der Tat ist eine geringe Retourenrate ein Indikator für eine hohe Kundenzufriedenheit und eine effektive Produktpräsentation. Es ist wichtig zu beachten, dass jede Branche unterschiedliche Retourenraten hat und ein Vergleich zu anderen Online-Händlern sinnvoll sein kann. Trotzdem bleibt es wichtig, die Ursachen für jede Rücksendung zu überwachen und zu analysieren, um kontinuierlich Verbesserungen vornehmen zu können.

In [205]:
#df drop Dexcription, quantity, InvoiceDate, UnitPrice ,Country	,TotalPrice
df = df.drop(['Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'Country', 'TotalPrice'], axis=1)
#CustomerID to int
df['CustomerID'] = df['CustomerID'].astype(int)
#Add a cloum with Retunrned and ad a 1 if in InvoiceNo is a C
df['Returned'] = np.where(df['InvoiceNo'].str.contains('C'), 1, 0)
df['Bought'] = np.where(df['InvoiceNo'].str.contains('C'), 0, 1)


In [206]:

#Group by CustomerID and sum the Bought and Returned
df = df.groupby(['CustomerID']).agg({'Bought': 'sum', 'Returned': 'sum'})

In [207]:
#drop all rows where Retundes is bigger than Bought
df = df[df['Returned'] <= df['Bought']]



In [208]:
df['ReturnRate'] =( df['Returned'] / df['Bought'])*100

In [209]:
#df drop rows where Returned is 0
df = df[df['Returned'] != 0]

# Chart Beschreibung

Um einen klaren Fokus auf die Datenpunkte zu schaffen, habe ich die Achsen leicht ausgegraut und die Labels nah an den Achsen positioniert, um zu verdeutlichen, dass sie zusammengehören. Durch die Entfernung des Rahmens und das Ausblenden des Grids wird die Ablenkung minimiert und der Betrachter kann sich voll und ganz auf die Datenpunkte konzentrieren. Dies erleichtert das Verständnis der Daten und verbessert die visuelle Übersicht.

In [212]:
chart5 = alt.Chart(df).mark_circle().encode(
    x=alt.X('Bought:Q',
             axis=alt.Axis(title='Purchases',
                           titleAnchor='start',
                           labelAngle=0,
                           grid=False)
        ),
    y=alt.Y('ReturnRate:Q',
            axis=alt.Axis(title='Return Rate',
                          titleAnchor='end',
                          labelAngle=0,
                          grid=False,)
                          ),
    color=alt.condition(
        alt.FieldOneOfPredicate("Purchases", [""]),
        alt.value("darkred"),
        alt.value("grey")
    ),
).properties(
    title={"text":["Returnrate compared to number of purchases"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"] },
    width=500,
    height=350
)

alt.layer(chart5).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=12,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
).transform_filter(
    (alt.datum.Bought < 500) 
)

alt.LayerChart(...)

# Conclusion 
Zusammenfassend lässt sich sagen, dass es einen negativen Zusammenhang zwischen der Anzahl der Einkäufe eines Kunden und seiner Returnrate gibt. Je mehr Einkäufe ein Kunde getätigt hat, desto niedriger ist seine Returnrate.

# Empfehlung

Eine Empfehlung für einen Online-Shop könnte sein, Kundenbelohnungsprogramme oder Rabatte für treue Kunden anzubieten, um ihre Bindung an den Shop zu stärken und ihre Returnrate zu verringern. Eine gute Kundenbetreuung kann ebenfalls dazu beitragen, die Zufriedenheit der Kunden zu erhöhen und somit die Returnrate zu verringern. Generell ist die Retunrate des Onlineshops sehr gering mit 2 % im Vergleich mit zu dem Durchschnitt von 10-15% (vgl. Repko 2022). 